# Ref

1. [如何 Dispatcher 里面注册算子](https://docs.pytorch.org/tutorials/advanced/dispatcher?utm_source=chatgpt.com)
2.

# PyTorch Dispatcher 入门到进阶（中文教程）

本笔记面向 **初学者**，用可运行的小例子，带你循序渐进理解：

1. Dispatcher 是什么、为什么需要它
2. 如何**查看调度表（dispatch table）**，理解“一个算子在不同模式/设备下调用哪个实现”
3. 用 `TorchDispatchMode` **拦截所有算子调用**，观察 dispatcher 实际路由
4. 认识 `vmap`（向量化 map）：为什么在 vmap 中，算子的行为会切换到 **Batched** 语义
5. 一些练习，帮助你加深理解

> 提示：本笔记使用到的部分 API（如 `_dispatch_dump_table`、`TorchDispatchMode`）属于 **内部/私有** 或 **实验性** 接口，在不同 PyTorch 版本可能有差异。建议使用 PyTorch 2.1 及以上版本。


## 0. 环境检查

查看 PyTorch 版本、CUDA 是否可用、首个 GPU 名称（如有）。


In [1]:
import torch
print('PyTorch 版本:', torch.__version__)
print('CUDA 可用:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU 设备:', torch.cuda.get_device_name(0))

PyTorch 版本: 2.7.1+cu126
CUDA 可用: True
GPU 设备: NVIDIA RTX A6000


## 1. 直观理解 Dispatcher：为什么需要它？

同一个算子（如 `add` 加法），在不同情境要走不同实现：

- **设备**：CPU vs CUDA（GPU）
- **是否需要梯度**：Autograd（反向传播）逻辑要插入/包装
- **vmap 模式**：算子应具备“批处理（Batched）”语义
- **tracing / 导出 / 混合精度** 等其他模式

如果都塞进一个函数里用 `if/else` 判断，代码会变得不可维护。**Dispatcher** 通过“**dispatch key** → **kernel**”的映射表，在运行时根据张量属性与上下文，选择合适实现。


## 2. 查看算子的调度表（Dispatch Table）

我们用 `_dispatch_dump_table` 打印 `aten::add` 与 `aten::mm` 的调度表，看看每个 **Dispatch Key** 对应哪个实现。


In [2]:
from torch import _C
print(_C._dispatch_dump_table('aten::add'))
print('\n' + '='*80 + '\n')
print(_C._dispatch_dump_table('aten::mm'))




Undefined: registered at /pytorch/build/aten/src/ATen/RegisterCompositeExplicitAutogradNonFunctional_0.cpp:7811 [default backend kernel]
CPU: registered at /pytorch/build/aten/src/ATen/RegisterCPU_0.cpp:3455 [kernel]
CUDA: registered at /pytorch/build/aten/src/ATen/RegisterCUDA_0.cpp:15875 [kernel]
HIP: registered at /pytorch/build/aten/src/ATen/RegisterCompositeExplicitAutogradNonFunctional_0.cpp:7811 [default backend kernel]
MPS: registered at /pytorch/build/aten/src/ATen/RegisterCompositeExplicitAutogradNonFunctional_0.cpp:7811 [default backend kernel]
IPU: registered at /pytorch/build/aten/src/ATen/RegisterCompositeExplicitAutogradNonFunctional_0.cpp:7811 [default backend kernel]
XPU: registered at /pytorch/build/aten/src/ATen/RegisterCompositeExplicitAutogradNonFunctional_0.cpp:7811 [default backend kernel]
HPU: registered at /pytorch/build/aten/src/ATen/RegisterCompositeExplicitAutogradNonFunctional_0.cpp:7811 [default backend kernel]
VE: registered at /pytorch/build/aten/src/

### 看懂输出的小贴士

- **行**（或区块）代表不同的 **Dispatch Key**（如 `CPU`, `CUDA`, `AutogradCPU`, `AutogradCUDA`, `Batched`/`FuncTorchBatched` 等）
- **目标**是该 key 下登记的 **kernel 实现**（可能是 C++/CUDA、或复合实现）
- 运行时，dispatcher 会依据 **优先级** 和 **当前上下文** 选用正确的 key → kernel


其中“vmap + dispatcher + batched 语义” 和你直接写 CUDA kernel／GPU 并行之间还有一些差别，它可以在高层（Python）写函数处理单样本，然后 vmap + dispatcher 自动把它“抬升”到批处理，无需自己管理并行细节。此外，vmap + dispatch key 的机制允许不同模式（Autograd、CUDA、CPU、Mixed Precision、Tracing 等）无缝组合，也允许控制流／索引／梯度等机制“不变或变化”。

在 PyTorch 内部，大部分算子（operator）都是先写好了针对不同后端 /不同模式（比如 CPU、CUDA、Autograd、vmap 等模式）的实现（kernel），然后通过 dispatcher 来决定在运行时用哪个实现。这些实现／kernel 会被注册到 dispatcher 的不同 Dispatch Key 下。每个 key 表示一种“模式”：例如 CPU, CUDA, AutogradCUDA, Batched, AutocastCUDA 等。这样一个算子就不是一个函数，而是一个调度表，每一个 key 对应一个实现。

当你在 Python 层调用算子（比如 x + y 或者 torch.add(x, y)），dispatcher 会做以下几步：

收集所有参与输入的张量的 dispatch key（每个张量都有自己的 dispatch key 集）

看当前线程／全局状态或模式，比如是否在 vmap 批量模式、是否 tracing、是否开启 autocast、是否需要 autograd 等

组合这些 key，排除某些 key（如果有 exclude set），再按优先级排序

从调度表里选第一个匹配的 kernel 执行


## 3. 用 `TorchDispatchMode` 拦截算子调用

我们用 `TorchDispatchMode` 写一个简单的 **日志模式**，打印每次算子调用的名称与张量信息。这样可以**直观看到** dispatcher 正在路由哪些 op。


In [ ]:
from torch.utils._python_dispatch import TorchDispatchMode
import torch

def _brief(x):
    if isinstance(x, torch.Tensor):
        return f"Tensor(device={x.device}, dtype={x.dtype}, shape={tuple(x.shape)}, requires_grad={x.requires_grad})"
    return repr(x)

class LoggingMode(TorchDispatchMode):
    def __torch_dispatch__(self, func, types, args=(), kwargs=None):
        kwargs = {} if kwargs is None else kwargs

        # 1) 优先：完整名（e.g. 'aten.add.Tensor'）
        op_full = None
        name_attr = getattr(func, "name", None)
        if callable(name_attr):
            try:
                op_full = name_attr()   # 关键：要调用！别忘了括号
            except Exception:
                op_full = None

        # 2) 退化：基础名 + 重载（e.g. 'add.Tensor'）
        if op_full is None:
            base = getattr(getattr(func, "overloadpacket", None), "__name__", None)
            overload = getattr(func, "overload", "default")
            if base is not None:
                op_full = f"{base}.{overload}"
            else:
                # 最保守的兜底
                op_full = str(func)

        arg_desc = ", ".join(_brief(a) for a in args)
        print(f"[DISPATCH] op={op_full} | args=[{arg_desc}]")
        return func(*args, **kwargs)

# 小测
x = torch.randn(3, 3)
y = torch.randn(3, 3)
with LoggingMode():
    z = x + y
    w = torch.mm(x, y)
print("结果形状:", z.shape, w.shape)


[DISPATCH] op=add.<bound method OpOverload.name of <OpOverload(op='aten.add', overload='Tensor')>> | args=[Tensor(device=cpu, dtype=torch.float32, shape=(3, 3), requires_grad=False), Tensor(device=cpu, dtype=torch.float32, shape=(3, 3), requires_grad=False)]
[DISPATCH] op=mm.<bound method OpOverload.name of <OpOverload(op='aten.mm', overload='default')>> | args=[Tensor(device=cpu, dtype=torch.float32, shape=(3, 3), requires_grad=False), Tensor(device=cpu, dtype=torch.float32, shape=(3, 3), requires_grad=False)]
结果形状: torch.Size([3, 3]) torch.Size([3, 3])


你应该能看到 `add` 与 `mm` 的 `[DISPATCH]` 日志。如果你在有 GPU 的机器上，把 `x = x.cuda(); y = y.cuda()`，再观察调用是否变化。


## 4. Autograd：带梯度的分发

当张量 `requires_grad=True` 时，dispatcher 会让运算走 **Autograd** 的分支（先做记录/包装，再落到设备实现）。下面演示：


In [ ]:
x = torch.randn(2, 3, requires_grad=True)
y = torch.randn(2, 3)
with LoggingMode():
    out = (x + y).sum()
    out.backward()  # 触发反向
print('x.grad 形状:', x.grad.shape)

观察日志：你会看到前向的 `add/sum`，以及触发反向传播后，Autograd 路径会调度相应的反向算子。


## 5. vmap：向量化语义如何影响分发

在 `vmap` 中，同样的 per-sample 函数会被**向量化**到 batch 维。内部通常通过激活 **Batched** dispatch key，让算子采用 batched 语义（而不是让你手写 Python 循环）。


In [ ]:
from torch import vmap

def per_sample_fn(t):
    return (t * t + torch.sin(t)).sum()

batch = torch.randn(5, 4, requires_grad=True)

# 5.1 手写循环（基准）
loop_out = torch.stack([per_sample_fn(batch[i]) for i in range(batch.size(0))])

# 5.2 vmap 版本（无显式 Python 循环）
batched_fn = vmap(per_sample_fn)
vmap_out = batched_fn(batch)

print('loop_vs_vmap 是否一致:', torch.allclose(loop_out, vmap_out))
print('输出形状:', loop_out.shape, vmap_out.shape)

### 5.3 观察 vmap 下的调度日志

把 `LoggingMode` 套在 vmap 调用外层，看看都触发了哪些 op。你会看到与 per-sample 函数里相同的算子，但它们会按 **Batched** 语义处理 batch 维。


In [ ]:
with LoggingMode():
    _ = batched_fn(batch)

## 6. 小练习（巩固）

1. 分别在 CPU 与 CUDA（如可用）上，比较 `aten::add` 的调度表输出有何差异。
2. 把张量改为 `requires_grad=True`，观察日志里出现的 Autograd 相关算子。
3. 修改 `per_sample_fn`，引入更多算子（如 `exp`、`log`、`matmul`），再用 vmap 对比“手写循环 vs vmap”的一致性与性能。
4. 进一步尝试 `in_dims`/`out_dims` 的不同设置，理解 vmap 的输入/输出批维规则。


## 7. 常见问答

**Q1：为什么我本机没有 `TorchDispatchMode`？**

- 你的 PyTorch 版本可能偏旧，或安装不完整。升级到新版本试试。

**Q2：`_dispatch_dump_table` 打印不出来？**

- 注意它是内部接口，某些版本可能隐藏/变更。也可以换打印别的算子（如 `aten::sin`）。

**Q3：vmap 和 DataLoader 有什么区别？**

- DataLoader 负责“**喂数据**”（批处理地加载样本），vmap 负责“**算子层面向量化**”（把对单样本的运算自动推广到批维，不必手写 for 循环）。


## 8. 参考资料（建议阅读）

- PyTorch Dispatcher 设计与背景（ezyang 博客）
- PyTorch 官方 Dispatcher Walkthrough（GitHub Wiki）
- `torch.vmap` 文档与教程
- `TorchDispatchMode` / 扩展 PyTorch 笔记
- 自定义算子 / `torch.library`（学习如何向调度表注册内核）

（说明：以上链接请参见对话中的“参考链接”列表）
